In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import numpy

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'

sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import re


In [ ]:
allImpactMuts = analysis_utils.load_in_df_with_progress(filePath = pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf', nLinesFile = 275000)


In [4]:
endometrialIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'all')
colorectalIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'all')
gliomaIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'all')

In [6]:
allImpactMuts['cancerType'] = allImpactMuts['Tumor_Sample_Barcode'].apply(lambda x:
        'Endometrial Cancer' if x in endometrialIds
        else 'Colorectal Cancer' if x in colorectalIds
        else 'Glioma' if x in gliomaIds
        else None
        )


In [7]:
selectMuts = allImpactMuts[allImpactMuts['cancerType'].notnull()]

In [9]:
relatedGenes = {
    'Endometrial Cancer': ["AKT1","APC","ARID1A","BCOR","CCND1","CDKN2A.p16INK4a","CTCF","CTNNB1",         
"ESR1","FBXW7","FGFR2","KRAS","NRAS","PIK3CA",
                           "PIK3R1","PPP2R1A","PTEN","RB1","RRAS2","SOX17","SPOP","TP53"],
    'Colorectal Cancer':["AMER1","APC","ARID1A","ASXL1","ATM","B2M","BRAF","CDKN2A.p16INK4a","CTNNB1","ELF3","EPHA3","ERBB3","FBXW7","JUN","KRAS","NRAS","PIK3CA","PIK3R1","PTEN","RBM10","RNF43","SMAD2","SMAD3","SMAD4","SOX9","TCF7L2","TP53"],
    'Glioma': ["ARID1A","ARID2","ATRX","BCOR","BRAF","CBL","CDKN1B","CDKN2A.p14arf","CDKN2A.p16INK4a","CDKN2C","CIC","DNMT3A","EGFR","FUBP1","H3F3A","IDH1","KRAS","NF1","NOTCH1","PDGFRA","PIK3CA","PIK3R1","PPM1D","PTEN","PTPN11","RB1","SETD2","STAG2","TP53"]
}

hyperGenes = {
    'Endometrial Cancer': ["ARID5B","B2M","CASP8","MAP3K1","MSH2","MSH6","NF1","RASA1"],
    'Colorectal Cancer': ["CASP8","HLA-A","HLA-B","MSH2"],
    'Glioma': ["MSH6"]
}

weaklyRelatedGenes = {
    'Endometrial Cancer':[ "ARID1B","ATM","ATR","CDKN1B","CDKN2A.p14arf","EPHA3","EPHA5","ERRFI1","INPPL1","JAK1","KMT2B","KMT2D","NFE2L2","ROS1","STAT3","ZFHX3"],
    'Colorectal Cancer':["ARID2","BMPR1A","CDKN1B","ERBB2","GNAS","MAP2K1","MAP2K4","PBRM1","PTCH1","RASA1","TGFBR2"],
    'Glioma':["FGFR1","HLA-B","PBRM1","SMARCB1","TSC1"]
}

In [15]:
tmbInfo = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/tmbInfo.tsv')
tmbDict = dict(zip(tmbInfo['Tumor_Sample_Barcode'], tmbInfo['Nmut_Mb']))

In [ ]:
listOfDicts = []
cntr = 0
for case in set(selectMuts['Tumor_Sample_Barcode']):
    nmut = selectMuts.shape[0]
    caseOncogenicMuts = selectMuts[(selectMuts['Tumor_Sample_Barcode'] == case) & (selectMuts['oncogenic'].notnull())]
    
    #GET TMB
    tmb = None
    if case in tmbDict:
        tmb = tmbDict[case]
    
    #get cancer type
    cancerType = None
    if case in endometrialIds: cancerType = 'Endometrial Cancer'
    if case in colorectalIds: cancerType = 'Colorectal Cancer'
    if case in gliomaIds: cancerType = 'Glioma'
     
    related = relatedGenes[cancerType]
    hyper = hyperGenes[cancerType]
    weak = weaklyRelatedGenes[cancerType]
    
    nRelatedMuts = caseOncogenicMuts[caseOncogenicMuts['Hugo_Symbol'].isin(related)].shape[0]
    nHyperMuts = caseOncogenicMuts[caseOncogenicMuts['Hugo_Symbol'].isin(hyper)].shape[0]
    nWeakMuts = caseOncogenicMuts[caseOncogenicMuts['Hugo_Symbol'].isin(weak)].shape[0]
    
    listOfDicts.append({'cancerType': cancerType, 'nRelated': nRelatedMuts, 'nHyperMuts': nHyperMuts,
    'nWeakMuts': nWeakMuts, 'nOncogenic':caseOncogenicMuts.shape[0], 'tmb': tmb})
        
    cntr += 1
    if cntr%100 == 0: print cntr
df = pd.DataFrame(listOfDicts)

In [24]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/geneMutTypeSummary.tsv', index=False, sep='\t')

**TRUNCATING MUTATIONS IN MSI, POLE ETC**

In [3]:
impactSigs = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/signatures_from_unfiltered_maf.txt')
impactSigs = mutationSigUtils.merge_signature_columns(impactSigs)
impactSigs['dominantSignature'] = impactSigs.apply(lambda row: 
        mutationSigUtils.get_dominant_signature(row.to_dict(), cols=None, prefix='mean', notEnoughMuts= True), axis=1)

casesWithMSISignature = set(impactSigs[(impactSigs['dominantSignature'] == 'mean_MMR')
                                      |((impactSigs['Nmut_Mb'] >= 30) & (impactSigs['dominantSignature'].isin(set(['mean_1']))))
                                        ]['Tumor_Sample_Barcode'])

casesWithPOLESignature = set(impactSigs[(impactSigs['dominantSignature'] == 'mean_10') & (impactSigs['Nmut_Mb'] > 30)]['Tumor_Sample_Barcode'])
casesWithTMZSignature = set(impactSigs[(impactSigs['dominantSignature'] == 'mean_11') & (impactSigs['Nmut_Mb'] > 30)]['Tumor_Sample_Barcode'])


In [11]:
msiMuts = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(casesWithMSISignature)]
poleMuts = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(casesWithPOLESignature)]

In [12]:
msiIndelMaf = msiMuts[msiMuts['Variant_Type'].isin(['INS', 'DEL'])]
poleTruncatingMaf = poleMuts[poleMuts['Consequence'] == 'stop_gained']

In [13]:
tumorSuppresors = set(['ERRFI1', 'ASXL2', 'PMAIP1', 'ACTG1', 'SUFU', 'FBXO11', 'MEN1', 'FAM58A', 'B2M', 'RB1', 'DUSP22', 'SESN1', 'GPS2', 'RAD51D', 'SMG1', 'CDC73', 'MAP3K1', 'SMARCB1', 'INPP4B', 'PARK2', 'SMAD4', 'CBFB', 'CDH1', 'PPP6C', 'SETDB1', 'SETDB2', 'NF2', 'CDKN2B', 'CDKN2C', 'CDKN2A', 'DDX3X', 'PIK3R1', 'BARD1', 'PDS5B', 'KLF4', 'SPRED1', 'VHL', 'SMAD2', 'PMS1', 'PMS2', 'SETD2', 'GATA3', 'TBL1XR1', 'MUTYH', 'SOCS1', 'FAM175A', 'ROBO1', 'ARID1B', 'ARID1A', 'TCF7L2', 'STK11', 'FOXA1', 'PTEN', 'FAT1', 'FAS', 'CYLD', 'MAX', 'SH2D1A', 'APC', 'NTHL1', 'CTCF', 'KDM5C', 'KMT2C', 'ZFHX3', 'FOXP1', 'PIGA', 'CDKN1B', 'CDKN1A', 'FUBP1', 'MSH2', 'ID3', 'TNFRSF14', 'TRAF3', 'EP400', 'BRIP1', 'ARID4A', 'ARID4B', 'XRCC2', 'DAXX', 'SDHAF2', 'ASXL1', 'AMER1', 'RASA1', 'EGR1', 'MST1', 'SOX17', 'RUNX1', 'PIK3R3', 'NCOR1', 'NF1', 'JAK1', 'PTPRD', 'CHEK2', 'CHEK1', 'SMC1A', 'TMEM127', 'STAG1', 'RAD51', 'TCF3', 'STAG2', 'ARID2', 'RAD50', 'RNF43', 'PARP1', 'BLM', 'CUX1', 'RECQL', 'RAD21', 'PTPN2', 'PTPN1', 'SLX4', 'INHA', 'PAX5', 'IRF1', 'TP53', 'HLA-A', 'IRF8', 'CBL', 'TOP1', 'SHQ1', 'PRDM1', 'NSD1', 'ATXN2', 'CREBBP', 'HDAC4', 'SESN2', 'PPP2R1A', 'EPHA7', 'ATM', 'EPHA3', 'POT1', 'SMAD3', 'MOB3B', 'TBX3', 'POLE', 'ATR', 'FANCD2', 'FH', 'BCORL1', 'SOX9', 'IKZF3', 'TSC1', 'TP63', 'MRE11A', 'SDHC', 'BTG1', 'POLD1', 'CIITA', 'SMC3', 'SAMHD1', 'RTEL1', 'ECT2L', 'PIK3R2', 'CRBN', 'FANCC', 'NBN', 'FANCA', 'HLA-B', 'RECQL4', 'DUSP4', 'ERCC2', 'FBXW7', 'TGFBR2', 'TGFBR1', 'MSH3', 'RBM15', 'TET1', 'TET3', 'SESN3', 'MGA', 'LTB', 'FOXL2', 'SH2B3', 'BCOR', 'HIST1H1D', 'ATRX', 'EP300', 'RAD51C', 'RAD51B', 'HIST1H1B', 'TNFAIP3', 'DICER1', 'ARID5B', 'LATS2', 'FOXO1', 'KEAP1', 'EZH2', 'SP140', 'NKX3-1', 'PBRM1', 'PALB2', 'CIC', 'BRCA1', 'DTX1', 'FLCN', 'SPEN', 'CD58', 'ERCC3', 'ERCC4', 'MSH6', 'BCL11B', 'BMPR1A', 'ERF', 'BRCA2', 'NOTCH2', 'EED', 'MITF', 'ELF3', 'SMARCA4', 'BBC3', 'ANKRD11', 'CEBPA', 'BCL2L11', 'AXIN2', 'AXIN1', 'CDK12', 'ESCO2', 'MLH1', 'SDHB', 'MED12', 'HNF1A', 'RYBP', 'ATP6V1B2', 'DNMT3B', 'KMT2B', 'KMT2A', 'DNMT3A', 'NFKBIA', 'TRAF5', 'KMT2D', 'SPOP', 'RBM10', 'P2RY8', 'TP53BP1', 'TSC2', 'KDM6A', 'EPCAM', 'PHOX2B', 'NPM1', 'BCL10', 'LATS1', 'HOXB13', 'ARID3A', 'PTPRT', 'PTPRS', 'INPPL1', 'NOTCH4', 'TET2', 'NOTCH1', 'CASP8', 'NOTCH3', 'GRIN2A', 'MAP2K4', 'WT1', 'BACH2', 'SDHA', 'BAP1', 'PTCH1', 'SDHD'])
msiIndelMaf['class'] = 'msi indel'
poleTruncatingMaf['class'] = 'pole stop gained'

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
combinedMaf = pd.concat([msiIndelMaf, poleTruncatingMaf])

In [44]:
combinedMaf['geneType'] = combinedMaf['Hugo_Symbol'].apply(lambda x: 'TSG' if x in tumorSuppresors else 'Oncogene')

In [45]:
combinedMafLim = combinedMaf[['geneType', 'Hugo_Symbol', 'class']]

In [48]:
#reduce the data to a tiny dataframe that we will plot
def summarize_data_for_plotting(df):
    dfPole = df[df['class'] == 'pole stop gained']
    dfMSI = df[df['class'] == 'msi indel']
    
    listOfDs = [{'fracClass': (1.0*dfPole[dfPole['geneType'] == 'TSG'].shape[0])/dfPole.shape[0], 'geneType': 'TSG', 'class': 'POLE STOP-GAIN'},
                {'fracClass': (1.0*dfPole[dfPole['geneType'] == 'Oncogene'].shape[0])/dfPole.shape[0], 'geneType': 'Oncogene', 'class': 'POLE STOP-GAIN'},
                {'fracClass': (1.0*dfMSI[dfMSI['geneType'] == 'TSG'].shape[0])/dfMSI.shape[0], 'geneType': 'TSG', 'class': 'MSI INDEL'},
                {'fracClass': (1.0*dfMSI[dfMSI['geneType'] == 'Oncogene'].shape[0])/dfMSI.shape[0], 'geneType': 'Oncogene', 'class': 'MSI INDEL'}
               ]
    return pd.DataFrame(listOfDs)
    

In [52]:
cdsInfo = analysis_utils.get_cds_size_targeted_by_impact(infoFilePath = pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/impact_gene_reference_signatures.tsv')

In [66]:
fracTSG = (1.0*sum([x[1] for x in cdsInfo.items() if x[0] in tumorSuppresors]))/sum(cdsInfo.values())
print fracTSG 

0.558026085931


In [49]:
finalDf = summarize_data_for_plotting(combinedMafLim)
finalDf.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/tsgFracsPOLE_MMR.tsv', index=False, sep='\t')